# Mini Project #1: Baseball Analytics

The overall purpose of this mini-project is to predicting MLB wins per season by modeling data to KMeans clustering model and linear regression models.

## Part 2: Further Data Handling

In this part, you are going to conduct further data handling tasks to prepare your data for the analysis. The data handling include, but not limited to: 
- __feature engineering__, 
- more __data visualization__, 
- more on __binning your continuous variables__, 
- __correlation analysis__ and __correlation based feature selection__.

Let's get started by importing the dependencies.

In [ ]:
import pandas as pd

# We will need `numpy` in this part - so let's import it as well
import numpy as np

Now let's read the data we processed in Part1, and continue working on it.

In [ ]:
df = pd.read_csv('../ba545-data/baseball_analytics_pt1.csv', header=0, index_col=0)

df.head()

In [1]:
## make sure you check the shape of `df` again


In [ ]:
## also check if there is any missing data in `df`


### Feature Engineering

Anyone who follows the game of baseball knows that, as Major League Baseball (MLB) progressed, different eras emerged where the amount of runs per game increased or decreased significantly. The dead ball era of the early 1900s is an example of a low scoring era and the steroid era at the turn of the 21st century is an example of a high scoring era. Hence, in this analysis, we want to __exclude all the game data before__ the year `1900`.

In [ ]:
#Filtering out the years before year 1900

#### Complete your code below
#### You can filter by the value of a certain column `col` in a dataframe `df` as:
#### df[df[col] > some_value]
#### then you want to save the filtered results as a new dataframe `df`


If you are into baseball, you will know Runs per Game (RPG) is an important factor in the game results (winning or not). Since we do not have that feature in the dataset, we are going to create that feaure from the features we have. In the dataset `df`, we have the runs (`df['R']`) and games (`df['G']`) features. We are going to use them to create RPG.

Firstly, we are doing the analysis on a yearly basis, so we need to aggregate the runs (`df['R']`) and games (`df['G']`) features to a yearly basis.

In [ ]:
# Aggregate runs and games data to a yearly basis

#### complete the code below
#### Aggregation is a very important technique in dataframes
#### `pandas` provides a good method called `groupby()` for that
#### since we want to aggregate to a yearly basis, we are going to use the feature `yearID`
#### and combine the `groupby()` results with `sum()` we will get the aggregated results

#### create a new pandas series called `runs_per_year`, 
#### which is aggregating runs (`df['R']`) on a yearly basis


#### do the same for games (`df['G']`)


#### We need to combine these two series into a dataframe ('rpg_df') to calculate RPG
#### We can do that by using the `pd.concat()` method provided by `pandas`
#### Since we want the two series are columns, we need to set the parameter `axis=1` in `concat()`


#### Now we can calculate RPG: which is very simple: rpg = runs_per_year/games_per_year


#### let's double check the values of `rpg_df` by looking at its first 10 values


### More on Data Visualization

Clearly, `rpg_df` is time series data - and in which the most important feature would be `rpg`. So we want to visulize it using line chart. `Matplotlib` has a useful method `plot()` for that purpose. We are going to use that for visualizing `rpg`.

First let's import `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')
# following statement set a large canvas for the visualization
# plt.rcParams["figure.figsize"] = (20,10)

In [ ]:

#### Complete the code below
#### use `plot()` method on `rpg_df`, and set y-axis as `rpg`.
#### the x-axis in this plot will be years, and you should notice that 
#### 'yearID' is the index of `rpg_df`
#### Instead of creating another column of `yearID` (which you can do),
#### you can simply tell `plot()` to use the index of the dataframe/series by
#### set the `use_index` parameter to `True`
#### set the `figsize` parameter to (16,8)


#### We want to set the title of the visualization as `MLB Yearly Runs Per Game`


#### Set the y-axis as `Runs per Game` use `.xlabel()`


#### The x-axis is year - so we set it as `Years`


#### show the plot


There is an alternative way of doing what we just did , which is shown as following.

In [ ]:
# Creating the runs per year and games per year 

## Use Goupby and sum the count yearly
df.groupby("yearID").sum()

runs_per_year = {}
games_per_year = {}

for i, row in df.iterrows():
    year=row['yearID']
    runs=row['R']
    games=row['G']
    if year in runs_per_year:
        runs_per_year[year]=runs_per_year[year]+ runs
        games_per_year[year]=games_per_year[year]+ games
    else:
        runs_per_year[year]=runs
        games_per_year[year]=games
        
## ADD notes to the print statement         
print('Runs_per_year', runs_per_year)
print(' Games_per_year', games_per_year)

With these two dictionaries created, we can do the calculation using them.

In [ ]:
# mlb runs per game (per year) dictionary 
runs_per_game = {}
for k,v in games_per_year.items():
    year = k 
    games = v
    runs = runs_per_year[year]
    runs_per_game[year]= runs/games
    
print(runs_per_game)

__Note__: dictionaries would be a good alternative of `pandas` functions.

Now let's visualize `runs_per_game` again using the dictionaries.

In [ ]:
# Creating lists from mlb_runs_per_game
## List of tuples (x, y) 
lists = sorted(runs_per_game.items())
x,y = zip(*lists)
# following statement set a large canvas for the visualization
plt.rcParams["figure.figsize"] = (16,8)


plt.plot(x,y)
plt.title("MLB Yearly Runs Per Game")
plt.xlabel("Year")
plt.ylabel("Runs per Game")
plt.legend('Runs_per_Game')
plt.show()

### More on Variable Binning

Remember in part 1 we binned wins (`df['W']`) to a categorical variable - that was for making the analysis a __classification__ problem. Sometimes we also using binning for __dimensionality reduction__ purposes. For instance, the feature year (`df['yearID']`) has more than 100 distinct values, so its dimensionality will be 100+. A lot of machine learning algorithms do not like high-dimensional features, so it is a common practice to __bin high-dimensional features__.

__Rule of thumb__: if a categorical feature has more than __5__ categories, you should consider bin it.

In the block below, you are going to create your own function (`assign_label`) and apply it to your data (`df`).

When a categorical variable is low-dimensional, we shoud create dummy variables for that. `pandas` provides a function called `get_dummies()` for that.

In [ ]:
# define the `assign_label` function

def assign_label(year):
    if year < 1920:
        return 1
    elif year >= 1920 and year <= 1941:
        return 2
    elif year >= 1942 and year <= 1945:
        return 3
    elif year >= 1946 and year <= 1962:
        return 4
    elif year >= 1963 and year <= 1976:
        return 5
    elif year >= 1977 and year <= 1992:
        return 6
    elif year >= 1993 and year <= 2009:
        return 7
    elif year >= 2010:
        return 8

#### complete the code below
#### Add `year_label` column to `df`
#### by applying `assign_label` to the `df['yearID'] column


#### create dummy variables for `year_label`
#### call `pd.get_dummies` on `df['year_label']`
#### set the `prefix` parameter of `get_dummies` to `'era'`
#### store the results in a dataframe `dummy_df`


#### Concatenate `df` and `dummy_df` using pd.concat()


#### Check whether the dummy variables are successfully created


We can add  *Runs per Game* data from the `runs_per_game` dictionary to `df`.

In [ ]:
# Create column for  runs per game from the runs_per_game dictionary
def assign_rpg(year):
    return runs_per_game[year]

df['rpg'] = df['yearID'].apply(assign_rpg)
df['rpg'].head()

An alternative way of binning the `yearID` feature is to bin it by **decade**, then create dummy variables based on **dacades**. This can be done using following code.

In [ ]:
# Convert years into decade bins and creating dummy variables
def assign_decade(year):
    if year < 1920:
        return 1910
    elif year >= 1920 and year <= 1929:
        return 1920
    elif year >= 1930 and year <= 1939:
        return 1930
    elif year >= 1940 and year <= 1949:
        return 1940
    elif year >= 1950 and year <= 1959:
        return 1950
    elif year >= 1960 and year <= 1969:
        return 1960
    elif year >= 1970 and year <= 1979:
        return 1970
    elif year >= 1980 and year <= 1989:
        return 1980
    elif year >= 1990 and year <= 1999:
        return 1990
    elif year >= 2000 and year <= 2009:
        return 2000
    elif year >= 2010:
        return 2010
    
#### complete the code below
#### Add `decade_label` column to `df`
#### by applying `assign_decade` function to the `df['yearID'] column


#### create dummy variables for `year_label`
#### call `pd.get_dummies` on `df['year_label']`
#### set the `prefix` parameter of `get_dummies` to `'era'`
#### store the results in a dataframe `decade_df`


#### Concatenate `df` and `decade_df` using pd.concat()


#### Check if above actions are done properly
#### by looking at the first 5 rows of `df`


The next step in __feature engineering__ is to remove all unwanted features, in this case, we used `yearID`, `year_level`, and `decade_level` to generate new dummy variables - thus we do not need them any more.

In [ ]:
#### complete your code here
df = df.drop(####)


_Runs per Game_ and _Runs Allowed per Game_ (at __game__ level, not aggregated) are important features in the baseball domain. Now let us go ahead and create them.

In [ ]:
#### Complete your code below
#### runs per game = # of runs/# of games
#### # of runs is in `df['R']`
#### # of games is in `df['G']`


#### runs allowed per game = # of runs allowed/# of games
#### # of runs allowed is in `df['RA']


#### double check whether our calculations are successful or not
#### by looking at the first 10 columns of `df`


In [ ]:
df.RA_per_game.describe()

### Correlation Analysis

Correlation analysis is an important topic in data analysis, which is also a basis of feature engineering. You want to select features that are __highly__ correlated with your target variable. 

__Note__: in the meanwhile, if you are using linear models, such as linear regression, decision tree, or naive Bayes, you do not want more than __one__ features are __highly__ correlated.

Correlation analysis can be done in several ways: one of the most direct is using the `corr()` method provided in `pandas`. In this part, we are going to calulate the correlation between a particular feature and the target using `numpy`, and then visualize the correlation trends.

In above analysis, we believe that **Runs per Game** and **Runs Allowed per Game** are two important features in predicting our target (`win`). We are going to visualize the correlation between each of them and `win` to prove that.

In [ ]:
# create a variable called `fig` which contains the size of a figure
fig = plt.figure(figsize=(12, 6))

# create two sub-figures, one for `runs per game`, one for `runs allowed per game`
ax1 = fig.add_subplot(1,2,1)
ax2 = fig.add_subplot(1,2,2)

# define the axis we use in the figure
x1 = df['R_per_game']
x2 = df['RA_per_game']
y = df['W']

#### Complete your code here
#### We are using scatterplot to visualize the correlation trend - which is the common practice
#### if you don't remember how to create a scatterplot, please refer to part 1
#### in the 1st sub-figure (`ax1`), we define the x-axis as 'Runs per Game'
#### and y-axis as 'Wins', and color as 'blue'

#### You will also add title as 'Runs per Game vs. Wins'

#### and then add proper axis names to it


# To better visualize the correlation trend, we are going to add trend line to it
# Following snippet add trend line of correlation to `ax1`

z = np.polyfit(x1, y, 1)
p = np.poly1d(z)
ax1.plot(x1,p(x1),"y--")

#### complete your code here
#### similarly, we are going to visualize the correlation between `runs allowed per game`
#### and `Wins` in `ax2`, we set the color to 'red'

#### You will also add title as 'Runs Allowed per Game vs. Wins'

#### you will then add x-axis label to `ax2` - note that you do not need 
#### to add y-axis label since it is shared with `ax1`


# We will use the similar code as above to add trend line in `ax2`
z1 = np.polyfit(x2, y, 1)
p1 = np.poly1d(z1)
ax2.plot(x2,p1(x2),"y--")

#### show the plot


Can you observe the different correlation trends in above visualization?

To see how each of the variables is correlated with the target variable, we will use the `corr()` method provided by `Pandas`.

In [ ]:
#### complete your code below
#### check if features are highly correlated with Wins(`df['W']`) (>.5)


You can observe some features are __highly__ correlated with Wins.

However, that is not the whole picture - for instance, `RA_per_game` is not highly correlated with Wins. In fact, a feature can be __highly__ and __negatively__ correlated with your target.

So the correct code should reflect that.

In [ ]:
#### complete your code below
#### check if features are highly correlated with Wins(`df['W']`) (>.5 or <-.5)


Now you can observe a few more are __highly__ correlated with Wins, such as `RA_per_game`.

Above code is equivalent to:

In [ ]:
s1 = df.corr()['W'] > 0.5 
s2 = df.corr()['W'] < -0.5
s1 | s2 # logical OR

Now we can create a new dataframe `data_features` that only contains useful features (excluding the target and unwanted features).

In [ ]:
#Creating a new dataframe excluding the target variable 

attributes = ['G','R','AB','H','2B','3B','HR','BB','SO','SB','RA','ER','ERA','CG',
             'SHO','SV','IPouts','HA','HRA','BBA','SOA','E','DP','FP','decade_1910','decade_1920',
              'decade_1930','decade_1940','decade_1950','decade_1960','decade_1970','decade_1980',
              'decade_1990','decade_2000','decade_2010','R_per_game','RA_per_game','rpg']

data_features = df[attributes]

# check the first 5 rows of `data_features`
data_features.head()

We can use following code to determine what columns are excluded in `data_features`.

In [ ]:
set(df.columns) - set(data_features.columns)

I believe we complete the data preparation phase of the analysis.

Let's save the processed data to CSV files so that we can re-use it in part 3.

In [ ]:
# save all features to a CSV file `baseball_analytics_features.csv`
data_features.to_csv('../ba545-data/baseball_analytics_features.csv')

# don't forget to save the target variable to another CSV file `baseball_analytics_target.csv`
df['Win_bins'].to_csv('../ba545-data/baseball_analytics_target.csv')

# we also want to save the # of wins in the original data for further analysis
df['W'].to_csv('../ba545-data/baseball_analytics_wins.csv')

That's all for part 2. Please make sure your sync the complete notebook to your github repo for submission.